In [1]:
import os

import pandas as pd
from bs4 import BeautifulSoup

In [2]:
path_artigos = os.path.join(os.getcwd(), 'artigos_brutos')

In [3]:
# contagem de artigos
soma = len(os.listdir(path_artigos))
print(f'Total de artigos: {soma}')

Total de artigos: 1681


# Criando o Dataset

In [8]:
dict_df = {'title': [], 'doi_id': [], 'file_bruto_path': [], 'tipo': [], 'ano': []}

for idx, arquivo in enumerate(os.listdir(path_artigos), start=1):
    print(f'Artigos lidos: {str(idx).rjust(4, "0")}', end='\r')

    path_abs  = os.path.join(path_artigos, arquivo)
    path_rel = os.path.relpath(os.path.join(path_artigos, arquivo))

    with open(path_abs, 'r', encoding='utf8') as f:
        texto_html = f.read()

    soup = BeautifulSoup(texto_html, 'lxml')

    try:
        doi_id = soup.find('a', attrs={'class': '_doi'}).text.strip().removeprefix('https://doi.org/')
    except AttributeError:
        doi_id = ''

    try:
        titulo = soup.find('h1', attrs={'class': 'article-title'}).text.strip()
    except AttributeError:
        titulo = ''

    try:
        tipo = soup.find('span', attrs={'class': '_articleBadge'}).text.strip()
    except AttributeError:
        tipo = ''

    try:
        ano = soup.find('span', attrs={'class': '_editionMeta'}).text.split('•')[-1].strip()
        if len(ano) > 4:
            ano = ano[-4:]
    except AttributeError:
        ano = ''

    dict_df['title'].append(titulo)
    dict_df['doi_id'].append(doi_id)
    dict_df['file_bruto_path'].append(path_rel)
    dict_df['tipo'].append(tipo)
    dict_df['ano'].append(ano)


In [26]:
df = pd.DataFrame(dict_df)

In [18]:
df.head()

,title,doi_id,file_bruto_path,tipo,ano
0,Mestrado Nacional Profissional em Ensino de Fí...,10.1590/1806-9126-RBEF-2021-0392,artigos_brutos\0000.html,Cartas ao Editor,2022
1,A equivalência entre o Princípio de Maximizaçã...,10.1590/1806-9126-RBEF-2021-0407,artigos_brutos\0001.html,Cartas ao Editor,2022
2,O impacto do MNPEF no desenvolvimento científi...,10.1590/1806-9126-RBEF-2022-0128,artigos_brutos\0002.html,Cartas ao Editor,2022
3,Quebra espontânea de simetria e mecanismo de H...,10.1590/1806-9126-RBEF-2021-0264,artigos_brutos\0003.html,Artigos Gerais,2022
4,Sombras de buracos negros: desvendando a físic...,10.1590/1806-9126-RBEF-2020-0232,artigos_brutos\0004.html,Artigos Gerais,2021


In [30]:
df['tipo'].value_counts()

Artigos Gerais                                                                          795
Produtos e Materiais Didáticos                                                          202
Pesquisa em Ensino de Física                                                            158
História da Física e Ciências Afins                                                     145
Desenvolvimento em Ensino de Física                                                      78
Notas e Discussões                                                                       61
Cartas ao Editor                                                                         27
Seção Especial: Informática no Ensino de Física                                          19
Editorial                                                                                17
Seção Especial                                                                           16
                                                                                

In [33]:
df[df['tipo'] == 'Tipo de artigo']

,title,doi_id,file_bruto_path,tipo,ano
400,Conceitos básicos de radiômetros de substituiç...,10.1590/1806-9126-RBEF-2018-0220,artigos_brutos\0400.html,Tipo de artigo,2019


In [34]:
df.to_csv('scielo_dataset.csv', index=False)